In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from classes import boundaries as bd
from classes import snag_MTF_func as sg

plot_setup = bd.plot_setup()
params = {'mathtext.default':'regular'}
plt.rcParams.update(params)

%matplotlib inline
%config Completer.use_jedi = False 

In [3]:
#---------------#
#  Import data  #
#---------------#


#--- Site MTF
site_mtf = sg.MTF_database_import('data/MTF_database.xlsx','Site_MTF')
site_mtf.loc[site_mtf['Site name'].isna(),'Site name'] = ''
site_mtf.loc[site_mtf['Y coords'].isna(),'Y coords'] = ''
site_mtf.loc[site_mtf['X coords'].isna(),'X coords'] = ''
site_mtf_cp = site_mtf.copy()

# Select data
site_mtf = site_mtf.groupby(['Reference','Continent','Country','Region','Site name','X coords','Y coords']
                                  ).first().reset_index()

#--- Species
species_mtf = sg.MTF_database_import('data/MTF_database.xlsx','Species_MTF')
species_mtf.loc[species_mtf['Site name'].isna(),'Site name'] = ''
species_mtf.loc[species_mtf['Y coords'].isna(),'Y coords'] = ''
species_mtf.loc[species_mtf['X coords'].isna(),'X coords'] = ''
species_mtf_cp = species_mtf.copy()

# Select data
species_mtf = species_mtf.groupby(['Reference','Continent','Country','Region','Site name', 'Species'],
                                  ).first().reset_index()

In [4]:
display(site_mtf_cp.groupby(['MTF basis','Mortality'])['MTF'].mean().round().to_frame())
display(species_mtf_cp.groupby(['MTF basis','Mortality'])['MTF'].mean().round().to_frame())

MTF
MTF basis Mortality      
c         Fire       20.0
          Other      10.0
cmass     Fire       58.0
          Insects    19.0
          Other      49.0
count     Fire       30.0
          Insects    10.0
          Other      26.0

MTF
MTF basis Mortality      
c         Fire       20.0
          Other       7.0
cmass     Fire       72.0
          Insects    18.0
          Other      27.0
count     Fire       47.0
          Insects    13.0
          Other      19.0

# Sites



In [5]:
# papers by continent
ss_site_continent = site_mtf.groupby(['Continent','Reference',]).count()['Country']
ss_site_continent = ss_site_continent.reset_index()
n_papers_continent = ss_site_continent.Continent.value_counts()

# sites by continent
n_sites_continent = site_mtf.Continent.value_counts()

# number of conifer | broadleave dominated sites
n_conifer_dominated_sites = site_mtf.groupby(['Continent'])['PFT'].sum()
site_mtf.loc[site_mtf['PFT'] == 1, 'PFT'] = 2
site_mtf.loc[site_mtf['PFT'] == 0, 'PFT'] = 1
site_mtf.loc[site_mtf['PFT'] == 2, 'PFT'] = 0
n_broadleaved_dominated_sites = site_mtf.groupby(['Continent'])['PFT'].sum()

# mean DBH
site_mean_DBH = site_mtf.groupby(['Continent'])['DBH mean'].mean().round(1)

# Fire 
site_n_Fire       = site_mtf.groupby(['Continent'])['Fire'].sum()
site_n_Management = site_mtf.groupby(['Continent'])['Managed'].sum()


#--- combine into dataframe
cols = pd.MultiIndex.from_tuples([('References', 'n'), 
                                  ('Sites','n'), 
                                  ('Sites','PFT'), 
                                  ('Sites','Fire'), 
                                  ('Sites','Managed'), 
                                  ('Sites','ø DBH')
                                 ])

df_sites = pd.concat([n_papers_continent, 
                      n_sites_continent, 
                      n_conifer_dominated_sites,
                      site_n_Fire,
                      site_n_Management,
                      site_mean_DBH
                     ],axis=1)
df_sites.columns = cols
all_row = {i:j for i,j in zip(cols, [df_sites.References.n.sum(), 
                                     df_sites.Sites.n.sum(), 
                                     n_conifer_dominated_sites.sum()/df_sites.Sites.n.sum()*(100), 
                                     site_n_Fire.sum()/df_sites.Sites.n.sum()*(100),
                                     site_n_Management.sum()/df_sites.Sites.n.sum()*(100),                                     
                                     site_mtf['DBH mean'].mean(),
                                   ])}
idx_vals = df_sites.index.tolist()+['All']
df_sites_all = df_sites.append(all_row,ignore_index=True)
df_sites_all.index = idx_vals
display(df_sites_all)

print('N studies with MTFc data: {}'.format(len(site_mtf[site_mtf['MTF basis'] == 'c'].Reference.unique())))
print('N studies with MTFcount data: {}'.format(len(site_mtf[site_mtf['MTF basis'] == 'count'].Reference.unique())))


References  Sites                                            
                       n      n        PFT       Fire    Managed      ø DBH
North America       57.0  113.0  95.000000  29.000000  27.000000  23.200000
Europe              10.0   15.0  14.000000   2.000000   6.000000  26.200000
Asia                 1.0    1.0   1.000000   0.000000   0.000000  27.600000
Oceania              1.0    1.0   0.000000   0.000000   1.000000        NaN
South America        1.0    1.0   0.000000   0.000000   0.000000        NaN
All                 70.0  131.0  83.969466  23.664122  25.954198  23.523177

N studies with MTFc data: 8
N studies with MTFcount data: 62


# Species

In [6]:
# species by continent
n_species_continent = species_mtf.Continent.value_counts()
n_species_PFTs_continent = species_mtf.groupby(['Continent'])['PFT'].sum()

# Only counting coniferous
species_mtf.loc[species_mtf['PFT'] == 1, 'PFT'] = 2
species_mtf.loc[species_mtf['PFT'] == 0, 'PFT'] = 1
species_mtf.loc[species_mtf['PFT'] == 2, 'PFT'] = 0
n_species_broadleaved_continent = species_mtf.groupby(['Continent'])['PFT'].sum()

# mean DBH
species_mean_DBH = species_mtf.groupby(['Continent'])['DBH mean'].mean().round(1)

# Fire 
spe_n_Fire       = species_mtf.groupby(['Continent'])['Fire'].sum()
spe_n_Management = species_mtf.groupby(['Continent'])['Managed'].sum()


df_species = pd.concat([n_species_continent, 
                        n_species_PFTs_continent,
                        spe_n_Fire,
                        spe_n_Management,
                        species_mean_DBH
                       ],axis=1)
cols = pd.MultiIndex.from_tuples([('Species','n'), 
                                  ('Species','PFT'), 
                                  ('Species','Fire'), 
                                  ('Species','Managed'),                                   
                                  ('Species','ø DBH')
                                 ])
df_species.columns = cols
all_row = pd.DataFrame({i:j for i,j in zip(cols, [df_species.Species.n.sum(), 
                                                  n_species_PFTs_continent.sum()/df_species.Species.n.sum()*(100), 
                                                  species_mtf['Fire'].mean()*(100),
                                                  species_mtf['Managed'].mean()*(100),                                     
                                                  species_mtf['DBH mean'].mean(),
                                   ])},index=['All'])
idx_vals = df_species.index.tolist()+['All']
df_species_all = pd.concat([df_species,all_row],ignore_index=True)
df_species_all.index = idx_vals
ss = pd.concat([df_sites_all, df_species_all], axis=1)

In [7]:
ss_dis = ss.copy()
#ss_dis.iloc[0,:] = ss.iloc[0,:]
#ss_dis.iloc[1,:] = ss.iloc[1,:]
#ss_dis.iloc[2,:] = ss.iloc[2,:]
#ss_dis.iloc[3,:] = ss.iloc[3,:]
#ss_dis.iloc[4,:] = ss.iloc[4,:]

#--- Sites
ss_dis.iloc[:-1,2] = ss.iloc[:-1,2] / ss.iloc[:-1,1] * 100 # PFT
ss_dis.iloc[:-1,3] = ss.iloc[:-1,3] / ss.iloc[:-1,1] * 100 # Fire
ss_dis.iloc[:-1,4] = ss.iloc[:-1,4] / ss.iloc[:-1,1] * 100 # Managed
#
#
##--- Species
## Conifer
ss_dis.iloc[:-1,7] = ss.iloc[:-1,7] / ss.iloc[:-1,6] * 100
ss_dis.iloc[:-1,8] = ss.iloc[:-1,8] / ss.iloc[:-1,6] * 100
ss_dis.iloc[:-1,9] = ss.iloc[:-1,9] / ss.iloc[:-1,6] * 100



#ss_dis[ss_dis.isna()] = '-'
ss_dis = ss_dis.infer_objects().round(1).copy()
display(ss_dis)


print(ss_dis.to_latex(na_rep='-',float_format="%.1f"))

References  Sites                            Species        \
                       n      n    PFT  Fire Managed ø DBH       n   PFT   
North America       57.0  113.0   84.1  25.7    23.9  23.2   153.0  68.6   
Europe              10.0   15.0   93.3  13.3    40.0  26.2    17.0  70.6   
Asia                 1.0    1.0  100.0   0.0     0.0  27.6     7.0  28.6   
Oceania              1.0    1.0    0.0   0.0   100.0   NaN     2.0   0.0   
South America        1.0    1.0    0.0   0.0     0.0   NaN     NaN   NaN   
All                 70.0  131.0   84.0  23.7    26.0  23.5   179.0  66.5   

                                   
               Fire Managed ø DBH  
North America  40.5    29.4  26.9  
Europe         11.8    70.6  23.6  
Asia            0.0     0.0  26.4  
Oceania         0.0   100.0   NaN  
South America   NaN     NaN   NaN  
All            35.8    33.0  26.8

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{} & References & \multicolumn{5}{l}{Sites} & \multicolumn{5}{l}{Species} \\
{} &          n &     n &   PFT & Fire & Managed & ø DBH &       n &  PFT & Fire & Managed & ø DBH \\
\midrule
North America &       57.0 & 113.0 &  84.1 & 25.7 &    23.9 &  23.2 &   153.0 & 68.6 & 40.5 &    29.4 &  26.9 \\
Europe        &       10.0 &  15.0 &  93.3 & 13.3 &    40.0 &  26.2 &    17.0 & 70.6 & 11.8 &    70.6 &  23.6 \\
Asia          &        1.0 &   1.0 & 100.0 &  0.0 &     0.0 &  27.6 &     7.0 & 28.6 &  0.0 &     0.0 &  26.4 \\
Oceania       &        1.0 &   1.0 &   0.0 &  0.0 &   100.0 &     - &     2.0 &  0.0 &  0.0 &   100.0 &     - \\
South America &        1.0 &   1.0 &   0.0 &  0.0 &     0.0 &     - &       - &    - &    - &       - &     - \\
All           &       70.0 & 131.0 &  84.0 & 23.7 &    26.0 &  23.5 &   179.0 & 66.5 & 35.8 &    33.0 &  26.8 \\
\bottomrule
\end{tabular}

